In [1]:
# pip install bitsandbytes accelerate flash-attn

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [3]:
import torch
import bz2
import pickle
import importlib
from tqdm import tqdm


import time
from IPython.display import JSON
import sys
sys.settrace(None)
import pdb

import copy

import psycopg
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TextStreamer, StoppingCriteria, StoppingCriteriaList, DynamicCache, OffloadedCache #, CodeGenTokenizer
from transformers.generation.logits_process import LogitsProcessorList

In [4]:
assert torch.cuda.is_available()

In [5]:
# index_module = importlib.import_module('phi4_index')
index_module = importlib.import_module('qwen25_index')
Index = getattr(index_module, 'Index')
index = Index()

# model_module = importlib.import_module('phi4_model')
model_module = importlib.import_module('qwen25_1B_model')
Model = getattr(model_module, 'Model')
model = Model()

Loading Qwen/Qwen2.5-1.5B-Instruct


Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


In [6]:
streamer = TextStreamer(model.tokenizer)

In [7]:
from ctrie import ConstrainedLogitsProcessor, ConstrainedStateList, \
                    ConstrainedState, DictIndex, GetAnswer

In [1]:
question = '''Which city is the capital of the country where the Eiffel Tower is?'''
question = '''Who was the quarterback of the team that won Super Bowl 50?'''
question = '''Who is the lead character in the Uncharted series?'''
question = '''Who played Gandalf in the Lord of the Rings movies?'''
question = '''What was Nirvana's last studio album called?''' # TODO test with beamsearch 3. with beam search if I have only 1 allowed token the other beams could generate something illegal and than crash
# TODO understand the crash is due to beam search and abandon those illegal paths. or try to explore always different triples with beam search???


question = '''How many members were there in the original NSYNC music group?'''

question = 'Who influenced Nirvana band?'

question = 'Is Johnny Depp or Brad Pitt older?'

prompted_texts = [model.apply_prompt_template(question)]

NameError: name 'model' is not defined

In [9]:
print(prompted_texts[0])

<|im_start|>system
You are a helpful question answering assistant that bases its answer on facts from a knowledge base.
1) You receive an input question.
2) You reason on the path you need to follow to reach the answer starting from the information in the question.
3) You explicitly provide relevant facts, one per line starting with "Fact:".
4) You explain your reasoning process and provide a long answer with your motivations based on the facts.
5) You provide a short concise answer.

Sometimes it could happen that there are no relevant facts in the knowledge base.
In these cases you either:
a) provide your own answer clearly stating that there were no supporting facts to your answer;
b) in case you don't know the answer or you are not confident enough, you just answer "I don't know".
<|im_end|>
<|im_start|>user
Which mountain is taller between Mont Blanc and Mount Rainier?
<|im_end|>
<|im_start|>assistant
Reasoning: I need to provide the height of Mont Blanc and the height of Mount Ra

In [2]:
"""prompted_texts = [
    '''
<|im_start|>system
You are a helpful question answering assistant that bases its answer on facts from a knowledge base.
1) You receive an input question.
2) You reason on the path you need to follow to reach the answer starting from the information in the question.
3) You explicitly provide relevant facts, one per line starting with "Fact:".
4) You explain your reasoning process and provide a long answer with your motivations based on the facts.
5) You provide a short concise answer.

Sometimes it could happen that there are no relevant facts in the knowledge base.
In these cases you either:
a) provide your own answer clearly stating that there were no supporting facts to your answer;
b) in case you don't know the answer or you are not confident enough, you just answer "I don't know".
<|im_end|>
<|im_start|>user
Which mountain is taller between Mont Blanc and Mount Rainier?
<|im_end|>
<|im_start|>assistant
Reasoning: I need to provide the height of Mont Blanc and the height of Mount Rainier, then I need to compare the two heights and the final answer will be the taller mountain.
Fact: <Mont Blanc> <elevation above sea level> <4,807.02±0.5 meters> .
Now I need the height of Mount Rainier.
Fact: <Mount Rainier> <elevation above sea level> <4,389 meters> .
Long answer: Mont Blanc is 4,807 meters tall, while Mount Rainier is 4,389 meters, so Mont Blanc is taller than Mount Rainier.

Answer: Mont Blanc.
<|im_end|>
<|im_start|>user
What is the fourth book in the Twilight series?<|im_end|>
<|im_start|>assistant
Reasoning: To answer the question about the fourth book in the Twilight series, I need to know the titles of the books in the series. I will search for the titles of the books in the Twilight series and identify the fourth one.
''']
prompted_texts = ['''<|im_start|>system
You are a helpful question answering assistant that bases its answer on facts from a knowledge base.
1) You receive an input question.
2) You reason on the path you need to follow to reach the answer starting from the information in the question.
3) You explicitly provide relevant facts, one per line starting with \"Fact:\".
4) You explain your reasoning process and provide a long answer with your motivations based on the facts.
5) You provide a short concise answer.

Sometimes it could happen that there are no relevant facts in the knowledge base.
In these cases you either:
a) provide your own answer clearly stating that there were no supporting facts to your answer;
b) in case you don't know the answer or you are not confident enough, you just answer \"I don't know\".
<|im_end|>
<|im_start|>user
Which mountain is taller between Mont Blanc and Mount Rainier?
<|im_end|>
<|im_start|>assistant
Reasoning: I need to provide the height of Mont Blanc and the height of Mount Rainier, then I need to compare the two heights and the final answer will be the taller mountain.
Fact: <Mont Blanc> <elevation above sea level> <4,807.02\u00b10.5 meters> .
Now I need the height of Mount Rainier.
Fact: <Mount Rainier> <elevation above sea level> <4,389 meters> .
Long answer: Mont Blanc is 4,807 meters tall, while Mount Rainier is 4,389 meters, so Mont Blanc is taller than Mount Rainier.

Answer: Mont Blanc.
<|im_end|>
<|im_start|>user
Is Johnny Depp or Brad Pitt older?<|im_end|>
<|im_start|>assistant




''']"""

'prompted_texts = [\n    \'\'\'\n<|im_start|>system\nYou are a helpful question answering assistant that bases its answer on facts from a knowledge base.\n1) You receive an input question.\n2) You reason on the path you need to follow to reach the answer starting from the information in the question.\n3) You explicitly provide relevant facts, one per line starting with "Fact:".\n4) You explain your reasoning process and provide a long answer with your motivations based on the facts.\n5) You provide a short concise answer.\n\nSometimes it could happen that there are no relevant facts in the knowledge base.\nIn these cases you either:\na) provide your own answer clearly stating that there were no supporting facts to your answer;\nb) in case you don\'t know the answer or you are not confident enough, you just answer "I don\'t know".\n<|im_end|>\n<|im_start|>user\nWhich mountain is taller between Mont Blanc and Mount Rainier?\n<|im_end|>\n<|im_start|>assistant\nReasoning: I need to provide

In [27]:
inputs = model.tokenizer(prompted_texts, return_tensors='pt', padding=True, padding_side='right')
inputs = inputs.to(model.device)

In [28]:
getanswer = GetAnswer(model.answer_tokens, model.eofanswer, all)
stopping_criteria = StoppingCriteriaList([
    getanswer
])

In [29]:
torch.cuda.empty_cache()

In [30]:
num_beams = 1

auto_streamer = streamer if num_beams == 1 else None

states = ConstrainedStateList(
    [ConstrainedState(
                begin_pattern = model.switch_pattern,
                end_pattern = model.newline_token,
                cache_index = DictIndex(end_of_triple=index.end_of_triple),
                subtree_cache = DictIndex(end_of_triple=index.end_of_triple),
                oneleaf_cache = DictIndex(end_of_triple=index.end_of_triple)
            ) for _ in range(num_beams)],
            num_beams=num_beams,
            batch_size = 1,
            pad_token_id = model.tokenizer.eos_token_id)

constrained_processor = ConstrainedLogitsProcessor(
    index=index.index,
    end_token=model.newline_token, states=states, tokenizer=model.tokenizer)
logits_processor_list = LogitsProcessorList([
    constrained_processor
])

model.model.eval()
start = time.time()

with torch.no_grad():

    getanswer.set_prompt(inputs.input_ids[0])

    #pdb.set_trace()
    out = model.model.generate(
        **inputs,
        logits_processor=logits_processor_list,
        max_new_tokens=200,
        streamer = auto_streamer,
        do_sample = False,
        temperature = 0.0,
        top_k=None,
        num_beams=num_beams,
        #num_beam_groups = num_beams,
        #diversity_penalty=0.5,
        #num_return_sequences=1,
        num_return_sequences=num_beams,
        #no_repeat_ngram_size=1,
        #remove_invalid_values=True,
        stopping_criteria=stopping_criteria,
        use_cache=True,
        #past_key_values=past_key_values,
        kwargs = {'constrained_state': states}, # passing state
    )

print('Elapsed', time.time() - start)

<|im_start|>system
You are a helpful question answering assistant that bases its answer on facts from a knowledge base.
1) You receive an input question.
2) You reason on the path you need to follow to reach the answer starting from the information in the question.
3) You explicitly provide relevant facts, one per line starting with "Fact:".
4) You explain your reasoning process and provide a long answer with your motivations based on the facts.
5) You provide a short concise answer.

Sometimes it could happen that there are no relevant facts in the knowledge base.
In these cases you either:
a) provide your own answer clearly stating that there were no supporting facts to your answer;
b) in case you don't know the answer or you are not confident enough, you just answer "I don't know".
<|im_end|>
<|im_start|>user
Which mountain is taller between Mont Blanc and Mount Rainier?
<|im_end|>
<|im_start|>assistant
Reasoning: I need to provide the height of Mont Blanc and the height of Mount Ra

In [19]:
model.tokenizer.decode([366, 22816, 34900, 29, 366, 3094, 29])

' <Twilight> <author>'

In [20]:
torch.cuda.empty_cache()

In [21]:
_from = len(inputs.input_ids[0]) # 0
for i in range(out.shape[0]):
    print('-'*30, sum(out[i][_from:]), len(out[i][_from:]))
    print(model.tokenizer.decode(out[i][_from:]))

------------------------------ tensor(1681951, device='cuda:0') 200
Fact: <Twilight> <author> <Fitz-Greene Halleck> .
Fact: <Twilight> <author> <Eduard von Keyserling> .
Fact: <Twilight> <author> <Paul Laurence Dunbar> .
Fact: <Twilight> <author> <Peter James (writer)> .
Fact: <Twilight> <author> <William Gay (author)> .
Fact: <Twilight> <author> <David R. George III> .
Fact: <Twilight> <author> <Henry Wadsworth Longfellow> .
Fact: <Twilight> <author> <Howard Chaykin> .
Now I need to find the titles of the books in the Twilight series.
Fact: <Twilight> <number of pages> <+498> .
Fact: <Twilight> <number of pages> <+504> .
Fact
------------------------------ tensor(1681951, device='cuda:0') 200
Fact: <Twilight> <author> <Fitz-Greene Halleck> .
Fact: <Twilight> <author> <Eduard von Keyserling> .
Fact: <Twilight> <author> <Paul Laurence Dunbar> .
Fact: <Twilight> <author> <Peter James (writer)> .
Fact: <Twilight> <author> <William Gay (author)> .
Fact: <Twilight> <author> <David R. George

In [22]:
for i, triple in enumerate(states[0].generated_triples):
    print(i, model.tokenizer.decode(triple)[:-1], triple, end='\n')

0  <Twilight> <author> <Fitz-Greene Halleck> . [366, 22816, 34900, 29, 366, 3094, 29, 366, 37, 11042, 12010, 265, 1952, 472, 5054, 377, 29, 659, 198]
1  <Twilight> <author> <Eduard von Keyserling> . [366, 22816, 34900, 29, 366, 3094, 29, 366, 36, 1054, 567, 6538, 5309, 799, 2718, 29, 659, 198]
2  <Twilight> <author> <Paul Laurence Dunbar> . [366, 22816, 34900, 29, 366, 3094, 29, 366, 25300, 32667, 16443, 28738, 2257, 29, 659, 198]
3  <Twilight> <author> <Peter James (writer)> . [366, 22816, 34900, 29, 366, 3094, 29, 366, 36559, 7801, 320, 18189, 16018, 659, 198]
4  <Twilight> <author> <William Gay (author)> . [366, 22816, 34900, 29, 366, 3094, 29, 366, 44787, 20677, 320, 3094, 16018, 659, 198]
5  <Twilight> <author> <David R. George III> . [366, 22816, 34900, 29, 366, 3094, 29, 366, 22286, 431, 13, 9857, 14429, 29, 659, 198]
6  <Twilight> <author> <Henry Wadsworth Longfellow> . [366, 22816, 34900, 29, 366, 3094, 29, 366, 63363, 467, 329, 60731, 5724, 69, 5299, 29, 659, 198]
7  <Twiligh

In [23]:
stop, ans = getanswer.get_answer(out[0])
# TODO: is getanswer still needed?
stop, model.tokenizer.decode(list(ans)) # TODO trim and remove special tokens

(False, '')